In [15]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [16]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [18]:
# get mean and std for each channel
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# intialize transformer
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
    ])

# load and transform data
train = datasets.CIFAR10(root='~/.pytorch/CIFAR10',train=True, download=True,transform=transform)
test = datasets.CIFAR10(root='~/.pytorch/CIFAR10',train=False, transform=transform)

train_loader = DataLoader(train, batch_size=64, shuffle=True)
test_loader = DataLoader(test, batch_size=64, shuffle=False)

Files already downloaded and verified


In [19]:
# load model
model = models.vgg16(pretrained=True)
model.classifier

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [20]:
# free layers
for param in model.parameters():
  param.requires_grad = False

In [21]:
# unfrezze fc classifier layers
for i in range(0,7):
  model.classifier[i].requires_grad = True

In [22]:
# modify last fc layer
model.classifier[6] = nn.Sequential(
                      nn.Linear(4096,512),
                      nn.ReLU(),
                      nn.Dropout(0.5),
                      nn.Linear(512,10),
                      nn.LogSoftmax(dim=1)
                      )

In [23]:
# unfreeze last two Conv layers
for i in range(17,24):
  model.features[i].requires_grad = True

In [24]:
# define learning rate, optimizer and loss
from torch.optim import Adam

criterion = nn.NLLLoss()

lr = 3e-4
optimizer = Adam([
    { 'params': model.features[17].parameters(), 'lr': lr/9},
    { 'params': model.features[19].parameters(), 'lr': lr/9},
    { 'params': model.features[21].parameters(), 'lr': lr/9},
    { 'params': model.features[24].parameters(), 'lr': lr/3},
    { 'params': model.features[26].parameters(), 'lr': lr/3},
    { 'params': model.features[28].parameters(), 'lr': lr/3},
    { 'params': model.classifier[0].parameters(), 'lr': lr},
    { 'params': model.classifier[3].parameters(), 'lr': lr},
    { 'params': model.classifier[6].parameters(), 'lr': lr}
    ], lr=lr)

In [27]:
model = model.to(device)

epochs = 2
batch_loss = 0
cum_epoch_loss = 0

for i in range(epochs):
  cum_epoch_loss = 0

  for batch_n, (X_train, y_train) in enumerate(train_loader,1):
    X_train = X_train.to(device)
    y_train = y_train.to(device)

    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()

    batch_loss += loss.item()
    if batch_n%100 == 0:
      print(f'Epoch({i+1}/{epochs}) : Batch no ({batch_n}/{len(train_loader)}) : Batch loss : {loss.item()}')

  print(f'Training loss : {batch_loss/len(train_loader)}')

Epoch(1/2) : Batch no (100/782) : Batch loss : 0.41212284564971924
Epoch(1/2) : Batch no (200/782) : Batch loss : 0.554536759853363
Epoch(1/2) : Batch no (300/782) : Batch loss : 0.5204486846923828
Epoch(1/2) : Batch no (400/782) : Batch loss : 0.49545422196388245
Epoch(1/2) : Batch no (500/782) : Batch loss : 0.3772347867488861
Epoch(1/2) : Batch no (600/782) : Batch loss : 0.3602065145969391
Epoch(1/2) : Batch no (700/782) : Batch loss : 0.28151190280914307
Training loss : 0.43393160589515706
Epoch(2/2) : Batch no (100/782) : Batch loss : 0.29285600781440735
Epoch(2/2) : Batch no (200/782) : Batch loss : 0.39718085527420044
Epoch(2/2) : Batch no (300/782) : Batch loss : 0.326334685087204
Epoch(2/2) : Batch no (400/782) : Batch loss : 0.22797878086566925
Epoch(2/2) : Batch no (500/782) : Batch loss : 0.41721826791763306
Epoch(2/2) : Batch no (600/782) : Batch loss : 0.2267834097146988
Epoch(2/2) : Batch no (700/782) : Batch loss : 0.4046380817890167
Training loss : 0.8046493748264849


### Get Test Accuracy

In [28]:
model.to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0

    #set_trace()
    for batch_n, (X_test, y_test) in enumerate(test_loader,1):

        y_pred = model(X_test)
        sfmax = torch.exp(y_pred)

        y_pred_class = torch.argmax(sfmax, 1)
        total += y_test.size(0)
        num_correct += (y_pred_class == y_test).sum().item()
        print(f'Batch ({batch_n}/{len(test_loader)})')

        if batch_n == 5:
          break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

Batch (1/157)
Batch (2/157)
Batch (3/157)
Batch (4/157)
Batch (5/157)
Accuracy of the model on 320 test images: 83.125% 
